In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge

# Parameters

In [71]:
X = np.array(
    [[1129, 1.435],
     [1453, 1.601],
     [1656, 1.654],
     [1787, 1.803],
     [1611, 1.734]]
)
y = [1.56, 1.64, 1.77, 1.83, 2.5]

X_test = np.array(
    [[1629, 1.635],
     [1853, 1.701],
     [1356, 1.454]]
)
lambd = 0.5

In [72]:
X_to_print = pd.DataFrame(X, columns = ['x_1','x_2'])
print(f'The original data (i.e., matrix X):')
X_to_print

The original data (i.e., matrix X):


,x_1,x_2
0,1129.0,1.435
1,1453.0,1.601
2,1656.0,1.654
3,1787.0,1.803
4,1611.0,1.734


#Standardize the continuous initial variables

In [73]:
X_std = (X - np.mean(X, axis=0)) / np.std(X, axis=0, ddof=1)
X_std_to_print = pd.DataFrame(X_std, columns = ['x_1_std','x_2_std'])
print(f'The standardised data (i.e., matrix X_std):')
print(f'x_1_std mean = {X_std_to_print["x_1_std"].mean():.2f}')
print(f'x_1_std std = {X_std_to_print["x_1_std"].std():.2f}')
print(f'x_2_std mean = {X_std_to_print["x_2_std"].mean():.2f}')
print(f'x_2_std std = {X_std_to_print["x_2_std"].std():.2f}')
X_std_to_print

The standardised data (i.e., matrix X_std):
x_1_std mean = -0.00
x_1_std std = 1.00
x_2_std mean = -0.00
x_2_std std = 1.00


,x_1_std,x_2_std
0,-1.576561,-1.497128
1,-0.293774,-0.315934
2,0.509947,0.061194
3,1.028605,1.121423
4,0.331782,0.630445


In [74]:
X_train_mean = np.mean(X, axis=0)
print(f'X_train_mean: \n{X_train_mean} \n')
X_train_std = np.std(X, axis=0, ddof=1)
print(f'X_train_std: \n{X_train_std} \n')

X_train_mean: 
[1527.2       1.6454] 

X_train_std: 
[2.52575137e+02 1.40535761e-01] 



#Standardize Y

In [75]:
y_train_mean = np.mean(y)
print(f'y_train_mean: \n{y_train_mean} \n')
y_train_std = np.std(y)
print(f'y_train_std: \n{y_train_std} \n')
y = [(a - y_train_mean)/y_train_std for a in y]
print(f'STD y: \n{y} \n')

y_train_mean: 
1.86 

y_train_std: 
0.3337663853655727 

STD y: 
[-0.8988322765679699, -0.6591436694831784, -0.26964968297039116, -0.08988322765679706, 1.9175088566783351] 



# Compute coefficients

In [76]:
# add the w_0 intercept where the corresponding x_0 = 1
Xp = np.concatenate([np.zeros((X_std.shape[0], 1)), X_std], axis=1)

# 2. the closed form solution
print(f'X: \n{Xp}\n')
print(f'XT: \n{Xp.T}\n')
print(f'labmda*I: \n{lambd * np.eye(Xp.shape[1])}\n')
print(f'Y: \n{y}\n')
print(f'XT*Y: \n{Xp.T.dot(y)}\n')
print(f'XTX+labmda*I: \n{Xp.T.dot(Xp) + lambd * np.eye(Xp.shape[1])}\n')
print(f'(XTX+labmda*I)^-1: \n{np.linalg.inv(Xp.T.dot(Xp) + lambd * np.eye(Xp.shape[1]))}\n')
beta = np.linalg.inv(Xp.T.dot(Xp) + lambd * np.eye(Xp.shape[1])).dot(Xp.T).dot(y)
print(f'beta: \n{beta} \n')

X: 
[[ 0.         -1.57656056 -1.49712784]
 [ 0.         -0.29377397 -0.31593382]
 [ 0.          0.50994726  0.06119439]
 [ 0.          1.02860481  1.12142276]
 [ 0.          0.33178246  0.63044452]]

XT: 
[[ 0.          0.          0.          0.          0.        ]
 [-1.57656056 -0.29377397  0.50994726  1.02860481  0.33178246]
 [-1.49712784 -0.31593382  0.06119439  1.12142276  0.63044452]]

labmda*I: 
[[0.5 0.  0. ]
 [0.  0.5 0. ]
 [0.  0.  0.5]]

Y: 
[-0.8988322765679699, -0.6591436694831784, -0.26964968297039116, -0.08988322765679706, 1.9175088566783351]

XT*Y: 
[0.         2.01693713 2.64549741]

XTX+labmda*I: 
[[0.5        0.         0.        ]
 [0.         4.5        3.84700302]
 [0.         3.84700302 4.5       ]]

(XTX+labmda*I)^-1: 
[[ 2.          0.          0.        ]
 [ 0.          0.82560207 -0.70579859]
 [ 0.         -0.70579859  0.82560207]]

beta: 
[ 0.         -0.20200087  0.76057675] 



# Ridge (Sklearn)

In [77]:
ridge = Ridge(fit_intercept=True, alpha=lambd)
ridge.fit(X_std, y)
# 1. intercept and coef of the fit model
coeff = np.array([ridge.intercept_] + list(ridge.coef_))
print([float(f'{c:.3f}') for c in coeff])

[-0.0, -0.202, 0.761]


# Predict (test set)

In [78]:
print(f'X_test: \n{X_test} \n')

X_test_std = (X_test - X_train_mean) / X_train_std
print(f'STD X_test: \n{X_test_std} \n')

Xp_test = np.concatenate([np.zeros((X_test_std.shape[0], 1)), X_test_std], axis=1)
print(f'Adding zeros STD X_test: \n{Xp_test} \n')

y_pred = Xp_test.dot(beta.T)
print(f'y_pred: \n{y_pred} \n')

print(f'y_pred restandertaization: \n{y_pred*y_train_std+y_train_mean} \n')

X_test: 
[[1.629e+03 1.635e+00]
 [1.853e+03 1.701e+00]
 [1.356e+03 1.454e+00]] 

STD X_test: 
[[ 0.40304838 -0.07400252]
 [ 1.28991319  0.39562884]
 [-0.6778181  -1.36193094]] 

Adding zeros STD X_test: 
[[ 0.          0.40304838 -0.07400252]
 [ 0.          1.28991319  0.39562884]
 [ 0.         -0.6778181  -1.36193094]] 

y_pred: 
[-0.13770072  0.04034251 -0.89893316] 

y_pred restandertaization: 
[1.81404013 1.87346497 1.55996633] 

